# pAPRika tutorial 5 - APR/Amber with Plumed restraints

In this tutorial, we will perform APR calculations for the butane (BUT)--cucurbit[6]uril (CB6) host-guest system. This is a repeat of [tutorial 1](01-tutorial-cb6-but.ipynb) using `Plumed`-based restraints and the `Amber` MD engine. `Plumed` is a plugin for MD codes that can analyze trajectories and perform free-energy calculations on collective variables. It is a versatile plugin that can interface with a number of MD engines. Here, we will go through the process of converting APR restraints constructed with pAPRika to a `Plumed` file and run a short calculation with `Sander`.

## <ins>Before you start</ins>
<a id='start'></a>
We will run the simulations in this tutorial using `sander` (Ambertools) and `Plumed`. Both of these should be installed in your `conda` environment if you installed **pAPRika** though the `conda` route. However, for `Plumed` to work with `Sander` we first need to make sure the `PLUMED_KERNEL` environment variable is loaded (the library is called `libplumedKernel.so`). **pAPRika** should load the `Plumed` kernel automatically but let's make sure it is loaded and run the cell below.

In [1]:
import os
'PLUMED_KERNEL' in os.environ.keys()

False

If it does not exists we will load the environment in this Jupyter Notebook. Since `Plumed` is installed through `conda` the kernel will be located in your conda environment library folder. If you are running this on a Mac replace the kernel library in the cell below to `libplumedKernel.dylib`. If you compiled `Plumed` yourself and then you will need to change the path below.

In [2]:
os.environ['PLUMED_KERNEL'] = f"{os.environ['CONDA_PREFIX']}/lib/libplumedKernel.so"

For running `Plumed` with Amber outside of this notebook it might be better to export the `PLUMED_KERNEL` variable into your `.bashrc` file.

<font size="4"><ins>**Note:**</ins></font> we can run `Plumed` with Amber versions 18 and 20, but version 18 requires you to patch the source code first and recompile. Older versions of Amber are not supported. See the `Plumed` documentation for more details https://www.plumed.org/doc-v2.6/user-doc/html/index.html.

🔵 Since we have a prepared the host-guest-dummy setup from the first tutorial, we will skip the initial tleap steps and go right into initializing the restraints.

Initialize logger

In [3]:
import logging
from importlib import reload
reload(logging)

logger = logging.getLogger()
logging.basicConfig(
    format='%(asctime)s %(message)s',
    datefmt='%Y-%m-%d %I:%M:%S %p',
    level=logging.INFO
)

### Define names
We will store the files created in this tutorial in a folder called `plumed` so we don't mix files with the previous tutorial.

In [4]:
base_name = "cb6-but-dum"
work_dir = "plumed"
complex_dir = "complex"

### Define anchor atoms
See [tutorial 1](01-tutorial-cb6-but.ipynb) for the choice of selection

In [5]:
# Guest atoms
G1 = ":BUT@C"
G2 = ":BUT@C3"

# Host atoms
H1 = ":CB6@C"
H2 = ":CB6@C31"
H3 = ":CB6@C18"

# Dummy atoms
D1 = ":DM1"
D2 = ":DM2"
D3 = ":DM3"

## Determine the number of windows
Before we add the restraints, it is helpful to set the $\lambda$ fractions that control the strength of the force constants during attach and release, and to define the distances for the pulling phase.

The attach fractions go from 0 to 1 and we place more points at the bottom of the range to sample the curvature of $dU/d \lambda$. Next, we generally apply a distance restraint until the guest is ~18 Angstroms away from the host, in increments of 0.4 Angstroms. This distance should be at least twice the Lennard-Jones cutoff in the system. These values have worked well for us, but this is one aspect that should be carefully checked for new systems.

In [6]:
import numpy as np

In [7]:
attach_string = "0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00"
attach_fractions = [float(i) / 100 for i in attach_string.split()]

initial_distance = 6.0
pull_distances = np.arange(0.0 + initial_distance, 18.0 + initial_distance, 1.0)

release_fractions = []

windows = [len(attach_fractions), len(pull_distances), len(release_fractions)]
logging.info(f"There are {windows} windows in this attach-pull-release calculation.")

2020-09-07 11:00:45 AM There are [15, 18, 0] windows in this attach-pull-release calculation.


## APR Restraints

In [8]:
import parmed as pmd

* Load complex structure

In [9]:
structure = pmd.load_file(
    os.path.join(complex_dir, f"{base_name}.prmtop"),
    os.path.join(complex_dir, f"{base_name}.rst7"),
    structure = True,
) 

### Host Static Restraints
See [tutorial 1](01-tutorial-cb6-but.ipynb#host_static) for an explanation of the static restraints

In [10]:
import paprika.restraints as restraints

2020-09-07 11:00:46 AM NumExpr defaulting to 8 threads.


In [11]:
static_restraints = []

In [12]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D1, H1],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 5.0,
                                    amber_index=False)

static_restraints.append(r)

In [13]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D2, D1, H1],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=False)

static_restraints.append(r)

In [14]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D3, D2, D1, H1],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=False)

static_restraints.append(r)

In [15]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D1, H1, H2],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=False)

static_restraints.append(r)

In [16]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D2, D1, H1, H2],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=False)

static_restraints.append(r)

In [17]:
r = restraints.static_DAT_restraint(restraint_mask_list = [D1, H1, H2, H3],
                                    num_window_list = windows,
                                    ref_structure = structure,
                                    force_constant = 100.0,
                                    amber_index=False)

static_restraints.append(r)

### Guest translational and rotational restraints
See [tutorial 1](01-tutorial-cb6-but.ipynb#guest) for an explanation of the guest restraints

In [18]:
guest_restraints = []

In [19]:
r = restraints.DAT_restraint()
r.mask1 = D1
r.mask2 = G1
r.topology = structure
r.auto_apr = True
r.continuous_apr = True
r.amber_index = False

r.attach["target"] = pull_distances[0]          # Angstroms
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 5.0                      # kcal/mol/Angstroms**2

r.pull["target_final"] = 24.0                   # Angstroms
r.pull["num_windows"] = windows[1]

r.initialize()
guest_restraints.append(r)

In [20]:
r = restraints.DAT_restraint()
r.mask1 = D2
r.mask2 = D1
r.mask3 = G1
r.topology = structure
r.auto_apr = True
r.continuous_apr = True
r.amber_index = False

r.attach["target"] = 180.0                      # Degrees
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 100.0                    # kcal/mol/radian**2

r.pull["target_final"] = 180.0                  # Degrees
r.pull["num_windows"] = windows[1]

r.initialize()
guest_restraints.append(r)

In [21]:
r = restraints.DAT_restraint()
r.mask1 = D1
r.mask2 = G1
r.mask3 = G2
r.topology = structure
r.auto_apr = True
r.continuous_apr = True
r.amber_index = False

r.attach["target"] = 180.0                      # Degrees
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 100.0                    # kcal/mol/radian**2

r.pull["target_final"] = 180.0                  # Degrees
r.pull["num_windows"] = windows[1]

r.initialize()
guest_restraints.append(r)

### Create APR windows
We use the guest restraints to create a list of windows with the appropriate names and then create the directories.

In [22]:
from paprika.restraints.restraints import create_window_list

In [23]:
window_list = create_window_list(guest_restraints)
window_phase = {**{x: "attach" for x in window_list if x[0] is 'a'}, 
                **{x: "pull" for x in window_list if x[0] is 'p'}}

2020-09-07 11:00:47 AM Restraints appear to be consistent


In [24]:
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
    
for window in window_list:
    folder = os.path.join(work_dir, window)
    if not os.path.isdir(folder):
        os.makedirs(os.path.join(work_dir, window))

## Write APR restraints to Plumed format
In this section we create an instance of `Plumed()` from `paprika.restraints.plumed`, which is a class to generate the `Plumed` restraint files. We need to specify the list of restraints used throughout the APR calculations and the corresponding windows list. In this tutorial we will print the **host static** restraints and the **guest** restraints. The `Plumed` class includes a method to add restraints to dummy atoms (`add_dummy_atoms_to_file`) but we will not do that here. Instead we will use the built-in position restraints feature in Amber (see [Simulation](#simulate) section below).

<font size='4'><ins>**Note**</ins></font>: be careful when specifiying the force constants in `DAT_restraints`. We follow the Amber (and CHARMM) convention where the force constant is already multiplied by a factor of 1/2 but `Plumed` requires the user to specify the force constant without this factor, i.e.

$$
\begin{align}
U_{amber} &= K_{amber} (r-r_{0})^2 \\
U_{plumed} &= \frac{1}{2} k_{plumed} (r - r_{0})^2 
\end{align}
$$

thus $k_{plumed} = 2 \times  K_{amber}$. If Amber force constants was used in generating the `DAT_restraints` (the case in this tutorial) we need to set the variable `uses_legacy_k` to `True` (this is on by default).

In [25]:
from paprika.restraints.plumed import Plumed

In [26]:
restraints_list = (static_restraints + guest_restraints)

plumed = Plumed()
plumed.file_name = 'plumed.dat'
plumed.path = work_dir
plumed.window_list = window_list
plumed.restraint_list = restraints_list
plumed.uses_legacy_k = True

plumed.dump_to_file()

## Translate guest molecule
For the attach windows, we will use the initial, bound coordinates for the host-guest complex. Only the force constants change during this phase, so a single set of coordinates is sufficient. For the pull windows, we will translate the guest to the target value of the restraint before solvation, and for the release windows, we will use the coordinates from the final pull window.

In [27]:
import shutil

In [28]:
for window in window_list:
    if window[0] == "a":
        shutil.copy(os.path.join(complex_dir, f"{base_name}.prmtop"),
                    os.path.join(work_dir, window, f"{base_name}.prmtop"))
        shutil.copy(os.path.join(complex_dir, f"{base_name}.rst7"),
                    os.path.join(work_dir, window, f"{base_name}.rst7"))

    elif window[0] == "p":
        structure = pmd.load_file(
            os.path.join(complex_dir, f"{base_name}.prmtop"), 
            os.path.join(complex_dir, f"{base_name}.rst7"), 
            structure = True
        )
        target_difference = guest_restraints[0].phase['pull']['targets'][int(window[1:])] -\
                            guest_restraints[0].pull['target_initial']
        logging.info(f"In window {window} we will translate the guest {target_difference:0.1f} Angstroms.")
        
        for atom in structure.atoms:
            if atom.residue.name == "BUT":
                atom.xz += target_difference
                
        structure.save(os.path.join(work_dir, window, f"{base_name}.prmtop"), overwrite=True)
        structure.save(os.path.join(work_dir, window, f"{base_name}.rst7"), overwrite=True)

2020-09-07 11:00:47 AM In window p000 we will translate the guest 0.0 Angstroms.
2020-09-07 11:00:47 AM In window p001 we will translate the guest 1.1 Angstroms.
2020-09-07 11:00:47 AM In window p002 we will translate the guest 2.1 Angstroms.
2020-09-07 11:00:47 AM In window p003 we will translate the guest 3.2 Angstroms.
2020-09-07 11:00:47 AM In window p004 we will translate the guest 4.2 Angstroms.
2020-09-07 11:00:47 AM In window p005 we will translate the guest 5.3 Angstroms.
2020-09-07 11:00:47 AM In window p006 we will translate the guest 6.4 Angstroms.
2020-09-07 11:00:47 AM In window p007 we will translate the guest 7.4 Angstroms.
2020-09-07 11:00:47 AM In window p008 we will translate the guest 8.5 Angstroms.
2020-09-07 11:00:47 AM In window p009 we will translate the guest 9.5 Angstroms.
2020-09-07 11:00:47 AM In window p010 we will translate the guest 10.6 Angstroms.
2020-09-07 11:00:47 AM In window p011 we will translate the guest 11.6 Angstroms.
2020-09-07 11:00:47 AM In 

## Simulate
<a id='simulate'></a>
Since we are going to run an implicit solvent simulation, we have everything ready to go. **pAPRika** has an `amber` module that can help setting default parameters for the simulation. There are some high level options that we set directly, like `simulation.path`, and then we call the function `config_gb_min()` to setup reasonable default simulation parameters for a minimization in the Generalized-Born ensemble. After that, we directly modify the simulation `cntrl` section to apply the positional restraints on the dummy atoms. 

We will run the simulations with `sander` but it is also possible and faster to run this with `pmemd` or `pmemd.cuda` if you have them installed.

<font size='4'><ins>**Note**</ins></font>: as explained at the [start](#start) of this tutorial, make sure that the `PLUMED_KERNEL` environment variable is set otherwise the simulation will fail to run.

In [29]:
from paprika import amber

Run a quick minimization in every window. Note that we need to specify `simulation.cntrl["ntr"] = 1` to enable the positional restraints on the dummy atoms.

### Energy Minimization

In [30]:
for window in window_list:
    simulation = amber.Simulation()
    simulation.executable = "sander"

    simulation.path = f"{work_dir}/{window}/"
    simulation.prefix = "minimize"

    simulation.inpcrd = "cb6-but-dum.rst7"
    simulation.ref = "cb6-but-dum.rst7"
    simulation.topology = "cb6-but-dum.prmtop"
    simulation.plumed_file = "plumed.dat"

    simulation.config_gb_min()
    simulation.cntrl["ntr"] = 1
    simulation.cntrl["restraint_wt"] = 50.0
    simulation.cntrl["restraintmask"] = "'@DUM'"

    logging.info(f"Running minimization in window {window}...")
    simulation.run(overwrite=True)

2020-09-07 11:00:48 AM Running minimization in window a000...
2020-09-07 11:00:48 AM Running Minimization at plumed/a000/
2020-09-07 11:00:54 AM Minimization completed...
2020-09-07 11:00:54 AM Running minimization in window a001...
2020-09-07 11:00:54 AM Running Minimization at plumed/a001/


KeyboardInterrupt: 

* Here we will skip the equilibration step and go straight to production!

### Production Run

In [31]:
for window in window_list:
    simulation = amber.Simulation()
    simulation.executable = "sander"
    
    simulation.path = f"{work_dir}/{window}/"
    simulation.prefix = "production"

    simulation.inpcrd = "minimize.rst7"
    simulation.ref = "cb6-but-dum.rst7"
    simulation.topology = "cb6-but-dum.prmtop"
    simulation.plumed_file = "plumed.dat"

    simulation.config_gb_md()
    simulation.cntrl["ntr"] = 1
    simulation.cntrl["restraint_wt"] = 50.0
    simulation.cntrl["restraintmask"] = "'@DUM'"
    
    logging.info(f"Running production in window {window}...")
    simulation.run(overwrite=True)

2020-09-07 11:00:59 AM Running production in window a000...
2020-09-07 11:00:59 AM Running MD at plumed/a000/
2020-09-07 11:01:06 AM STDOUT/STDERR received from AMBER execution
2020-09-07 11:01:06 AM b'PLUMED: PLUMED is starting'
2020-09-07 11:01:06 AM b'PLUMED: Version: 2.6.1 (git: Unknown) compiled on Jul  8 2020 at 08:08:27'
2020-09-07 11:01:06 AM b'PLUMED: Please cite these papers when using PLUMED [1][2]'
2020-09-07 11:01:06 AM b'PLUMED: For further information see the PLUMED web page at http://www.plumed.org'
2020-09-07 11:01:06 AM b'PLUMED: Root: /home/peanut/.conda/envs/paprika-dev/lib/plumed'
2020-09-07 11:01:06 AM b'PLUMED: For installed feature, see /home/peanut/.conda/envs/paprika-dev/lib/plumed/src/config/config.txt'
2020-09-07 11:01:06 AM b'PLUMED: Molecular dynamics engine: amber'
2020-09-07 11:01:06 AM b'PLUMED: Precision of reals: 8'
2020-09-07 11:01:06 AM b'PLUMED: Running over 1 node'
2020-09-07 11:01:06 AM b'PLUMED: Number of threads: 1'
2020-09-07 11:01:06 AM b'PLU

2020-09-07 11:01:06 AM b'PLUMED:   and linear force constant 0.000000'
2020-09-07 11:01:06 AM b'PLUMED:   added component to this action:  @18.force2 '
2020-09-07 11:01:06 AM b'PLUMED: END FILE: plumed.dat'
2020-09-07 11:01:06 AM b'PLUMED: Timestep: 0.000002'
2020-09-07 11:01:06 AM b'PLUMED: KbT: 0.592481'
2020-09-07 11:01:06 AM b'PLUMED: Relevant bibliography:'
2020-09-07 11:01:06 AM b'PLUMED:   [1] The PLUMED consortium, Nat. Methods 16, 670 (2019)'
2020-09-07 11:01:06 AM b'PLUMED:   [2] Tribello, Bonomi, Branduardi, Camilloni, and Bussi, Comput. Phys. Commun. 185, 604 (2014)'
2020-09-07 11:01:06 AM b'PLUMED: Please read and cite where appropriate!'
2020-09-07 11:01:06 AM b'PLUMED: Finished setup'
2020-09-07 11:01:06 AM b'PLUMED:                                               Cycles        Total      Average      Minumum      Maximum'
2020-09-07 11:01:06 AM b'PLUMED:                                                    1     0.302550     0.302550     0.302550     0.302550'
2020-09-07 11

2020-09-07 11:01:12 AM b'PLUMED:   with label @15'
2020-09-07 11:01:12 AM b'PLUMED:   with arguments c6'
2020-09-07 11:01:12 AM b'PLUMED:   added component to this action:  @15.bias '
2020-09-07 11:01:12 AM b'PLUMED:   at -1.107000'
2020-09-07 11:01:12 AM b'PLUMED:   with harmonic force constant 200.000000'
2020-09-07 11:01:12 AM b'PLUMED:   and linear force constant 0.000000'
2020-09-07 11:01:12 AM b'PLUMED:   added component to this action:  @15.force2 '
2020-09-07 11:01:12 AM b'PLUMED: Action RESTRAINT'
2020-09-07 11:01:12 AM b'PLUMED:   with label @16'
2020-09-07 11:01:12 AM b'PLUMED:   with arguments c7'
2020-09-07 11:01:12 AM b'PLUMED:   added component to this action:  @16.bias '
2020-09-07 11:01:12 AM b'PLUMED:   at 6.000000'
2020-09-07 11:01:12 AM b'PLUMED:   with harmonic force constant 0.040000'
2020-09-07 11:01:12 AM b'PLUMED:   and linear force constant 0.000000'
2020-09-07 11:01:12 AM b'PLUMED:   added component to this action:  @16.force2 '
2020-09-07 11:01:12 AM b'PLUME

2020-09-07 11:01:18 AM b'PLUMED:   at 2.573400'
2020-09-07 11:01:18 AM b'PLUMED:   with harmonic force constant 200.000000'
2020-09-07 11:01:18 AM b'PLUMED:   and linear force constant 0.000000'
2020-09-07 11:01:18 AM b'PLUMED:   added component to this action:  @11.force2 '
2020-09-07 11:01:18 AM b'PLUMED: Action RESTRAINT'
2020-09-07 11:01:18 AM b'PLUMED:   with label @12'
2020-09-07 11:01:18 AM b'PLUMED:   with arguments c3'
2020-09-07 11:01:18 AM b'PLUMED:   added component to this action:  @12.bias '
2020-09-07 11:01:18 AM b'PLUMED:   at 0.103900'
2020-09-07 11:01:18 AM b'PLUMED:   with harmonic force constant 200.000000'
2020-09-07 11:01:18 AM b'PLUMED:   and linear force constant 0.000000'
2020-09-07 11:01:18 AM b'PLUMED:   added component to this action:  @12.force2 '
2020-09-07 11:01:18 AM b'PLUMED: Action RESTRAINT'
2020-09-07 11:01:18 AM b'PLUMED:   with label @13'
2020-09-07 11:01:18 AM b'PLUMED:   with arguments c4'
2020-09-07 11:01:18 AM b'PLUMED:   added component to thi

2020-09-07 11:01:25 AM b'PLUMED:   with label c5'
2020-09-07 11:01:25 AM b'PLUMED:   between atoms 124 123 1 92'
2020-09-07 11:01:25 AM b'PLUMED:   without periodic boundary conditions'
2020-09-07 11:01:25 AM b'PLUMED: Action TORSION'
2020-09-07 11:01:25 AM b'PLUMED:   with label c6'
2020-09-07 11:01:25 AM b'PLUMED:   between atoms 123 1 92 55'
2020-09-07 11:01:25 AM b'PLUMED:   without periodic boundary conditions'
2020-09-07 11:01:25 AM b'PLUMED: Action DISTANCE'
2020-09-07 11:01:25 AM b'PLUMED:   with label c7'
2020-09-07 11:01:25 AM b'PLUMED:   between atoms 123 109'
2020-09-07 11:01:25 AM b'PLUMED:   without periodic boundary conditions'
2020-09-07 11:01:25 AM b'PLUMED: Action ANGLE'
2020-09-07 11:01:25 AM b'PLUMED:   with label c8'
2020-09-07 11:01:25 AM b'PLUMED:   between atoms 124 123 109'
2020-09-07 11:01:25 AM b'PLUMED:   without periodic boundary conditions'
2020-09-07 11:01:25 AM b'PLUMED: Action ANGLE'
2020-09-07 11:01:25 AM b'PLUMED:   with label c9'
2020-09-07 11:01:25 

...

2020-09-07 11:04:03 AM b'PLUMED:   and linear force constant 0.000000'
2020-09-07 11:04:03 AM b'PLUMED:   added component to this action:  @13.force2 '
2020-09-07 11:04:03 AM b'PLUMED: Action RESTRAINT'
2020-09-07 11:04:03 AM b'PLUMED:   with label @14'
2020-09-07 11:04:03 AM b'PLUMED:   with arguments c5'
2020-09-07 11:04:03 AM b'PLUMED:   added component to this action:  @14.bias '
2020-09-07 11:04:03 AM b'PLUMED:   at -2.159600'
2020-09-07 11:04:03 AM b'PLUMED:   with harmonic force constant 200.000000'
2020-09-07 11:04:03 AM b'PLUMED:   and linear force constant 0.000000'
2020-09-07 11:04:03 AM b'PLUMED:   added component to this action:  @14.force2 '
2020-09-07 11:04:03 AM b'PLUMED: Action RESTRAINT'
2020-09-07 11:04:03 AM b'PLUMED:   with label @15'
2020-09-07 11:04:03 AM b'PLUMED:   with arguments c6'
2020-09-07 11:04:03 AM b'PLUMED:   added component to this action:  @15.bias '
2020-09-07 11:04:03 AM b'PLUMED:   at -1.107000'
2020-09-07 11:04:03 AM b'PLUMED:   with harmonic for

2020-09-07 11:04:09 AM b'PLUMED: Action RESTRAINT'
2020-09-07 11:04:09 AM b'PLUMED:   with label @10'
2020-09-07 11:04:09 AM b'PLUMED:   with arguments c1'
2020-09-07 11:04:09 AM b'PLUMED:   added component to this action:  @10.bias '
2020-09-07 11:04:09 AM b'PLUMED:   at 9.295500'
2020-09-07 11:04:09 AM b'PLUMED:   with harmonic force constant 10.000000'
2020-09-07 11:04:09 AM b'PLUMED:   and linear force constant 0.000000'
2020-09-07 11:04:09 AM b'PLUMED:   added component to this action:  @10.force2 '
2020-09-07 11:04:09 AM b'PLUMED: Action RESTRAINT'
2020-09-07 11:04:09 AM b'PLUMED:   with label @11'
2020-09-07 11:04:09 AM b'PLUMED:   with arguments c2'
2020-09-07 11:04:09 AM b'PLUMED:   added component to this action:  @11.bias '
2020-09-07 11:04:09 AM b'PLUMED:   at 2.573400'
2020-09-07 11:04:09 AM b'PLUMED:   with harmonic force constant 200.000000'
2020-09-07 11:04:09 AM b'PLUMED:   and linear force constant 0.000000'
2020-09-07 11:04:09 AM b'PLUMED:   added component to this a

2020-09-07 11:04:15 AM b'PLUMED: Action ANGLE'
2020-09-07 11:04:15 AM b'PLUMED:   with label c2'
2020-09-07 11:04:15 AM b'PLUMED:   between atoms 124 123 1'
2020-09-07 11:04:15 AM b'PLUMED:   without periodic boundary conditions'
2020-09-07 11:04:15 AM b'PLUMED: Action TORSION'
2020-09-07 11:04:15 AM b'PLUMED:   with label c3'
2020-09-07 11:04:15 AM b'PLUMED:   between atoms 125 124 123 1'
2020-09-07 11:04:15 AM b'PLUMED:   without periodic boundary conditions'
2020-09-07 11:04:15 AM b'PLUMED: Action ANGLE'
2020-09-07 11:04:15 AM b'PLUMED:   with label c4'
2020-09-07 11:04:15 AM b'PLUMED:   between atoms 123 1 92'
2020-09-07 11:04:15 AM b'PLUMED:   without periodic boundary conditions'
2020-09-07 11:04:15 AM b'PLUMED: Action TORSION'
2020-09-07 11:04:15 AM b'PLUMED:   with label c5'
2020-09-07 11:04:15 AM b'PLUMED:   between atoms 124 123 1 92'
2020-09-07 11:04:15 AM b'PLUMED:   without periodic boundary conditions'
2020-09-07 11:04:15 AM b'PLUMED: Action TORSION'
2020-09-07 11:04:15 A

2020-09-07 11:04:15 AM Running MD at plumed/p017/
2020-09-07 11:04:22 AM STDOUT/STDERR received from AMBER execution
2020-09-07 11:04:22 AM b'PLUMED: PLUMED is starting'
2020-09-07 11:04:22 AM b'PLUMED: Version: 2.6.1 (git: Unknown) compiled on Jul  8 2020 at 08:08:27'
2020-09-07 11:04:22 AM b'PLUMED: Please cite these papers when using PLUMED [1][2]'
2020-09-07 11:04:22 AM b'PLUMED: For further information see the PLUMED web page at http://www.plumed.org'
2020-09-07 11:04:22 AM b'PLUMED: Root: /home/peanut/.conda/envs/paprika-dev/lib/plumed'
2020-09-07 11:04:22 AM b'PLUMED: For installed feature, see /home/peanut/.conda/envs/paprika-dev/lib/plumed/src/config/config.txt'
2020-09-07 11:04:22 AM b'PLUMED: Molecular dynamics engine: amber'
2020-09-07 11:04:22 AM b'PLUMED: Precision of reals: 8'
2020-09-07 11:04:22 AM b'PLUMED: Running over 1 node'
2020-09-07 11:04:22 AM b'PLUMED: Number of threads: 1'
2020-09-07 11:04:22 AM b'PLUMED: Cache line size: 512'
2020-09-07 11:04:22 AM b'PLUMED: 

2020-09-07 11:04:22 AM b'PLUMED:   added component to this action:  @18.force2 '
2020-09-07 11:04:22 AM b'PLUMED: END FILE: plumed.dat'
2020-09-07 11:04:22 AM b'PLUMED: Timestep: 0.000002'
2020-09-07 11:04:22 AM b'PLUMED: KbT: 0.592481'
2020-09-07 11:04:22 AM b'PLUMED: Relevant bibliography:'
2020-09-07 11:04:22 AM b'PLUMED:   [1] The PLUMED consortium, Nat. Methods 16, 670 (2019)'
2020-09-07 11:04:22 AM b'PLUMED:   [2] Tribello, Bonomi, Branduardi, Camilloni, and Bussi, Comput. Phys. Commun. 185, 604 (2014)'
2020-09-07 11:04:22 AM b'PLUMED: Please read and cite where appropriate!'
2020-09-07 11:04:22 AM b'PLUMED: Finished setup'
2020-09-07 11:04:22 AM b'PLUMED:                                               Cycles        Total      Average      Minumum      Maximum'
2020-09-07 11:04:22 AM b'PLUMED:                                                    1     0.306159     0.306159     0.306159     0.306159'
2020-09-07 11:04:22 AM b'PLUMED: 1 Prepare dependencies                          500

## Analysis

Once the simulation is completed, we can using the `analysis` module to determine the binding free energy. We supply the location of the parameter information, a string or list for the file names (wildcards supported), the location of the windows, and the restraints on the guest.

In this example, we use the method `ti-block` which determines the free energy using **t**hermodynamic **i**integration and then estimates the standard error of the mean at each data point using blocking analysis. Bootstrapping it used to determine the uncertainty of the  full thermodynamic integral for each phase.

After running `compute_free_energy()`, a dictionary called `results` will be populated, that contains the free energy and SEM for each phase of the simulation.

In [ ]:
import paprika.analysis as analysis

In [ ]:
free_energy = analysis.fe_calc()
free_energy.prmtop = "cb6-but-dum.prmtop"
free_energy.trajectory = 'production*.nc'
free_energy.path = work_dir
free_energy.restraint_list = guest_restraints
free_energy.collect_data()
free_energy.methods = ['ti-block']
free_energy.ti_matrix = "full"
free_energy.bootcycles = 1000
free_energy.compute_free_energy()

We also need to calculate the free-energy cost of releasing the restraints on the guest molecule.

In [ ]:
free_energy.compute_ref_state_work([
    guest_restraints[0], guest_restraints[1], None, None,
    guest_restraints[2], None
])

Then we add the free-energies together and combine the uncertainties to get the binding-free energy

In [ ]:
binding_affinity = -1 * (
    free_energy.results["attach"]["ti-block"]["fe"] + \
    free_energy.results["pull"]["ti-block"]["fe"] + \
    free_energy.results["ref_state_work"]
)

sem = np.sqrt(
    free_energy.results["attach"]["ti-block"]["sem"]**2 + \
    free_energy.results["pull"]["ti-block"]["sem"]**2
)

In [ ]:
print(f"The binding affinity of butane to cucurbit[6]uril = {binding_affinity:0.2f} +/- {sem:0.2f} kcal/mol")